# Семинар по курсу "Методы анализа данных"

## Векторизованное представление и обработка текстов

В качестве примера рассмотрим задачу [анализа тональности текста.](https://ru.wikipedia.org/wiki/%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7_%D1%82%D0%BE%D0%BD%D0%B0%D0%BB%D1%8C%D0%BD%D0%BE%D1%81%D1%82%D0%B8_%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%B0)

Будем использовать набор данных [Sentiment Labelled Sentences Data Set.](https://www.kaggle.com/marklvl/sentiment-labelled-sentences-data-set/version/2)

In [1]:
import numpy as np
import pandas as pd
from typing import Dict, Tuple
from scipy import stats
from IPython.display import Image
from sklearn.datasets import load_iris, load_boston
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, median_absolute_error, r2_score 
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.svm import SVC, NuSVC, LinearSVC, OneClassSVM, SVR, NuSVR, LinearSVR
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
sns.set(style="ticks")

In [2]:
def accuracy_score_for_classes(
    y_true: np.ndarray, 
    y_pred: np.ndarray) -> Dict[int, float]:
    """
    Вычисление метрики accuracy для каждого класса
    y_true - истинные значения классов
    y_pred - предсказанные значения классов
    Возвращает словарь: ключ - метка класса, 
    значение - Accuracy для данного класса
    """
    # Для удобства фильтрации сформируем Pandas DataFrame 
    d = {'t': y_true, 'p': y_pred}
    df = pd.DataFrame(data=d)
    # Метки классов
    classes = np.unique(y_true)
    # Результирующий словарь
    res = dict()
    # Перебор меток классов
    for c in classes:
        # отфильтруем данные, которые соответствуют 
        # текущей метке класса в истинных значениях
        temp_data_flt = df[df['t']==c]
        # расчет accuracy для заданной метки класса
        temp_acc = accuracy_score(
            temp_data_flt['t'].values, 
            temp_data_flt['p'].values)
        # сохранение результата в словарь
        res[c] = temp_acc
    return res

def print_accuracy_score_for_classes(
    y_true: np.ndarray, 
    y_pred: np.ndarray):
    """
    Вывод метрики accuracy для каждого класса
    """
    accs = accuracy_score_for_classes(y_true, y_pred)
    if len(accs)>0:
        print('Метка \t Accuracy')
    for i in accs:
        print('{} \t {}'.format(i, accs[i]))

## Загрузка данных

In [3]:
imdb_df = pd.read_csv("data/imdb_labelled.txt", delimiter='\t', header=None, names=['text', 'value'])
imdb_df.head()

,text,value
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [4]:
imdb_df.shape

(748, 2)

In [5]:
imdb_df['value'].unique()

array([0, 1], dtype=int64)

In [6]:
# Сформируем общий словарь для обучения моделей из обучающей и тестовой выборки
vocab_list = imdb_df['text'].tolist()
vocab_list[1:10]

['Not sure who was more lost - the flat characters or the audience, nearly half of whom walked out.  ',
 'Attempting artiness with black & white and clever camera angles, the movie disappointed - became even more ridiculous - as the acting was poor and the plot and lines almost non-existent.  ',
 'Very little music or anything to speak of.  ',
 'The best scene in the movie was when Gerardo is trying to find a song that keeps running through his head.  ',
 "The rest of the movie lacks art, charm, meaning... If it's about emptiness, it works I guess because it's empty.  ",
 'Wasted two hours.  ',
 'Saw the movie today and thought it was a good effort, good messages for kids.  ',
 'A bit predictable.  ',
 'Loved the casting of Jimmy Buffet as the science teacher.  ']

In [7]:
vocabVect = CountVectorizer()
vocabVect.fit(vocab_list)
corpusVocab = vocabVect.vocabulary_
print('Количество сформированных признаков - {}'.format(len(corpusVocab)))

Количество сформированных признаков - 3047


In [8]:
for i in list(corpusVocab)[1:10]:
    print('{}={}'.format(i, corpusVocab[i]))

slow=2404
moving=1750
aimless=92
movie=1748
about=37
distressed=748
drifting=786
young=3037
man=1639


## Векторная модель слов и n-грамм 

Векторизация текста поддерживается библиотекой [scikit-learn.](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)

### 1) Для векторизации можно использовать простой класс [CountVectorizer.](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) Подсчитывает количество слов словаря, входящих в данный текст.

In [9]:
test_features = vocabVect.transform(vocab_list)

In [10]:
test_features

<748x3047 sparse matrix of type '<class 'numpy.int64'>'
	with 11363 stored elements in Compressed Sparse Row format>

In [11]:
test_features.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [12]:
# Размер нулевой строки
len(test_features.todense()[0].getA1())

3047

In [13]:
# Непустые значения нулевой строки
[i for i in test_features.todense()[0].getA1() if i>0]

[1, 1, 1, 1, 1, 1, 1, 1, 3, 1]

In [14]:
vocabVect.get_feature_names()[100:120]

['all',
 'allison',
 'allow',
 'allowing',
 'almost',
 'along',
 'alongside',
 'already',
 'also',
 'although',
 'always',
 'am',
 'amateurish',
 'amaze',
 'amazed',
 'amazing',
 'amazingly',
 'america',
 'american',
 'americans']

### Использование [N-грамм](https://ru.wikipedia.org/wiki/N-%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B0)

В компьютерной лингвистике существует гипотеза о том, что основными носителями смысла в предложении являются не слова, а словосочетания. Поэтому существует возможность объединения соседних слов в N-граммы. 

In [15]:
ncv = CountVectorizer(ngram_range=(1,3))
ngram_features = ncv.fit_transform(vocab_list)
ngram_features

<748x24445 sparse matrix of type '<class 'numpy.int64'>'
	with 36257 stored elements in Compressed Sparse Row format>

In [16]:
len(ncv.get_feature_names())

24445

In [17]:
# Теперь признаками являются N-граммы
ncv.get_feature_names()[1000:1020]

['and adorable',
 'and adorable the',
 'and aesthetically',
 'and aesthetically like',
 'and all',
 'and all costs',
 'and all horror',
 'and all study',
 'and all the',
 'and along',
 'and along with',
 'and amazing',
 'and amazing dee',
 'and an',
 'and an excellent',
 'and an uplifting',
 'and angel',
 'and angel underneath',
 'and anita',
 'and anita laselva']

### Использование [нормализации текста](https://en.wikipedia.org/wiki/Text_normalization)

Проблемой рассмотренного выше подхода является то, что признаками могут становиться "мусорные" слова и симоволы (точка, запятая, прелоги, союзы), которые встречаются в каждом преложении и не характеризуют особенности текста. Также слово может встречаться с различными окончаниями, в различных падежах и т.д. 

Для более корректного формирования признаков на основе слов используются [стемминг](https://ru.wikipedia.org/wiki/%D0%A1%D1%82%D0%B5%D0%BC%D0%BC%D0%B8%D0%BD%D0%B3) и [лемматизация](https://ru.wikipedia.org/wiki/%D0%9B%D0%B5%D0%BC%D0%BC%D0%B0%D1%82%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F).

В питоне можно использовать библиотеки https://www.nltk.org/ и https://pymorphy2.readthedocs.io/en/latest/

Очень хорошей подборкой ресурсов по компьютерной лингвистике на русском языке является сайт https://nlpub.ru/

### 2) Для векторизации можно использовать более сложный класс [TfidfVectorizer.](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) Вычисляет специфичность текста в корпусе текстов на основе метрики [TF-IDF](https://ru.wikipedia.org/wiki/TF-IDF).

In [18]:
tfidfv = TfidfVectorizer(ngram_range=(1,3))
tfidf_ngram_features = tfidfv.fit_transform(vocab_list)
tfidf_ngram_features

<748x24445 sparse matrix of type '<class 'numpy.float64'>'
	with 36257 stored elements in Compressed Sparse Row format>

In [19]:
tfidf_ngram_features.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [20]:
# Размер нулевой строки
len(tfidf_ngram_features.todense()[0].getA1())

24445

In [21]:
# Непустые значения нулевой строки
[i for i in tfidf_ngram_features.todense()[0].getA1() if i>0]

[0.10376972549441718,
 0.1805172501755286,
 0.1805172501755286,
 0.1805172501755286,
 0.1805172501755286,
 0.1805172501755286,
 0.1805172501755286,
 0.1805172501755286,
 0.1805172501755286,
 0.1805172501755286,
 0.1805172501755286,
 0.1805172501755286,
 0.1338145998369827,
 0.07092090504272837,
 0.16994870261450212,
 0.1805172501755286,
 0.15663390735295346,
 0.1805172501755286,
 0.1805172501755286,
 0.1478636759209861,
 0.16994870261450212,
 0.1805172501755286,
 0.2898493852585766,
 0.16994870261450212,
 0.1805172501755286,
 0.32490039757353784,
 0.1805172501755286,
 0.1805172501755286,
 0.15663390735295346,
 0.1805172501755286]

### 3) Для векторизации текста можно использовать семантические признаки.

Основными подходами являются:
- [Word2Vec.](https://ru.wikipedia.org/wiki/Word2vec) Реализация для Питона - https://radimrehurek.com/gensim/models/word2vec.html
- [GloVe.](https://en.wikipedia.org/wiki/GloVe_(machine_learning))

[Детальный пример](https://www.kaggle.com/pierremegret/gensim-word2vec-tutorial) использования Word2Vec. 

## Решение задачи анализа тональности

С использованием кросс-валидации попробуем применить к корпусу текстов различные варианты векторизации и классификации.

In [22]:
def VectorizeAndClassify(vectorizers_list, classifiers_list):
    for v in vectorizers_list:
        for c in classifiers_list:
            pipeline1 = Pipeline([("vectorizer", v), ("classifier", c)])
            score = cross_val_score(pipeline1, imdb_df['text'], imdb_df['value'], scoring='accuracy', cv=3).mean()
            print('Векторизация - {}'.format(v))
            print('Модель для классификации - {}'.format(c))
            print('Accuracy = {}'.format(score))
            print('===========================')

In [23]:
vectorizers_list = [CountVectorizer(vocabulary = corpusVocab), TfidfVectorizer(vocabulary = corpusVocab)]
classifiers_list = [LogisticRegression(C=3.0), LinearSVC(), KNeighborsClassifier()]
VectorizeAndClassify(vectorizers_list, classifiers_list)

Векторизация - CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None,
                vocabulary={'10': 0, '12': 1, '13': 2, '15': 3, '15pm': 4,
                            '17': 5, '18th': 6, '1928': 7, '1947': 8, '1948': 9,
                            '1949': 10, '1971': 11, '1973': 12, '1980': 13,
                            '1986': 14, '1995': 15, '1998': 16, '20': 17,
                            '2005': 18, '2006': 19, '20th': 20, '25': 21,
                            '30': 22, '40': 23, '50': 24, '54': 25, '70': 26,
                            '70000': 27, '70s': 28, '80': 29, ...})
Модель для классификации - LogisticRegression(C=3.0, class_we

### Разделим выборку на обучающую и тестовую и проверим решение для лучшей модели

In [24]:
X_train, X_test, y_train, y_test = train_test_split(imdb_df['text'], imdb_df['value'], test_size=0.5, random_state=1)

In [25]:
def sentiment(v, c):
    model = Pipeline(
        [("vectorizer", v), 
         ("classifier", c)])
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print_accuracy_score_for_classes(y_test, y_pred)

In [26]:
sentiment(TfidfVectorizer(), LogisticRegression(C=5.0))

Метка 	 Accuracy
0 	 0.7964071856287425
1 	 0.6859903381642513


In [27]:
sentiment(TfidfVectorizer(ngram_range=(1,3)), LogisticRegression(C=5.0))

Метка 	 Accuracy
0 	 0.8143712574850299
1 	 0.6763285024154589
